 To-do
 
 > dimensionality reduction
 > model with features from cnn and with features from images
 
 
 - [Data exploration](#section-one)
 - [The model](#section-two)
 - [What went wrong/right](#section-three)


#### please comment if you have any improvements

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

#### This is a copy of my kernel for the Kannada mnist competition, to see how the model works with a different data set

https://www.kaggle.com/jakelj/kannada-mnist-beginner-to

In [ ]:
#Import Packages

import random
import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as mpatches

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from math import sqrt


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input, BatchNormalization
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')



<a name="section-one"></a>
## Data Exploration

In [ ]:
train.head()

In [ ]:
y_train = train['label']
X = train.drop(['label'],axis=1)


In [ ]:
X.shape

In [ ]:
test.shape

Almost half the number of samples in the test set

In [ ]:


plt.imshow(X.values[3].reshape(28,28))



The number 4

In [ ]:
# code for this plot taken from https://www.kaggle.com/josephvm/kannada-with-pytorch

fig, ax = plt.subplots(nrows=10, ncols=10, figsize=(15,15))

# I know these for loops look weird, but this way num_i is only computed once for each class
for i in range(10): # Column by column
    num_i = X[y_train == i]
    ax[0][i].set_title(i)
    for j in range(10): # Row by row
        ax[j][i].axis('off')
        ax[j][i].imshow(num_i.iloc[j, :].to_numpy().astype(np.uint8).reshape(28, 28))


by eye, it looks as if there may be some issues distinguishing between 5 and 6 on occasion

In [ ]:
import seaborn as sns
lab, val = np.unique(y_train,return_counts=True)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,7))
sns.barplot(lab,val)
plt.title('distribution of samples in the training data')

let's see if the number of pixels that are non '0' are informative to the label? I suspect some numbers require more 'ink' than others

In [ ]:
labels =[]
frequencies = []


for i in range(len(y_train)):
    lab, freq = str(y_train[i]), len([n for n in X.values[i] if n > 0])
    labels.append(lab)
    frequencies.append(freq)
    
    
data = {'Labels':labels, 'Frequencies':frequencies}

df = pd.DataFrame(data)

In [ ]:
# mean number of pixels per label
df.groupby('Labels').mean()

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
sns.boxplot(x = 'Labels', y = 'Frequencies', data = df)


The digits in this dataset are more unifrom in those of the kannada dataset. Although the number of non zero pixles are lower in that dataset, I wonder if it is due to the nature of the symbols or the images.

I wonder if the digits of the image give us any usefull information?

In [ ]:
# apply threshold
image = X.values[3].reshape(28,28)
thresh = threshold_otsu(image)
bw = closing(image > thresh, square(3))

# label image regions
label_image = label(bw)
image_label_overlay = label2rgb(label_image, image=image)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for region in regionprops(label_image):
    # take regions with large enough areas
    if region.area >= 30:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='green', linewidth=2)
        ax.add_patch(rect)
plt.text(1,1, f'Width: {maxc-minc} Height: {maxr -minr} Diagonal: {round(sqrt((((maxc-minc)**2) + (maxr-minr)**2)),2)}', color = 'w')
ax.set_axis_off()
plt.tight_layout()




In [ ]:
def measurements(images):
    
    widths = []
    heights = []
    diags = []
    
    for i in range(len(images)):
        # apply threshold
        image = images[i].reshape(28,28)
        thresh = threshold_otsu(image)
        bw = closing(image > thresh, square(3))

        # label image regions
        label_image = label(bw)
        image_label_overlay = label2rgb(label_image, image=image)

        for region in regionprops(label_image):
            # take regions with large enough areas
            if region.area >= 30:
                # draw rectangle around segmented coins
                minr, minc, maxr, maxc = region.bbox
                
        widths.append(maxc-minc)
        heights.append(maxr -minr)
        diags.append(sqrt((((maxc-minc)**2) + (maxr-minr)**2)))
                     
    return widths, heights, diags
                     


widths, heights, diags = measurements(X.values)
                     
data = {'Labels':labels, 'diagonals':diags, 'widths':widths, 'heights':heights, 'Area':frequencies}

df = pd.DataFrame(data)


In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
sns.boxplot(x = 'Labels', y = 'diagonals', data = df)



In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
sns.boxplot(x = 'Labels', y = 'heights', data = df)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
sns.boxplot(x = 'Labels', y = 'widths', data = df)

So, what have I learnt? Well, the height is very similar for each digit. This is strange to me as I would have though handwriting would vary alot more than that. Perhaps there is a logic error in my code?

the width of the digit is much more variable but still, its not a huge difference.

Let's see if a decision tree can tell the difference given these data

In [ ]:
from sklearn.ensemble import RandomForestClassifier
test_y = df['Labels']

test_x = df
test_x.drop('Labels', axis =1, inplace = True)


In [ ]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(test_x, test_y, test_size=0.1, random_state=1337)

In [ ]:
clf_rfc = RandomForestClassifier(n_estimators = 100)
clf_rfc.fit(X_train1,y_train1)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(clf_rfc.predict(X_val1), y_val1)

Wow I guess that is why CNN's are popular for this kind of problem.

let's try a KNN for fun before we move on

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


clf_knn = KNeighborsClassifier()
clf_knn.fit(X_train1,y_train1)

accuracy_score(clf_knn.predict(X_val1), y_val1)

Nope still not great.

This kind of model and these features are not going to cut it.

Seeing as everyone else is doing so I assume it is well worth learning...

#### Dimensionality reduction

In my laymens terms, what a PCA does is find the variables that are responsible for the most amount of variation in the dataset. If we had a straight line all the data would be in one component (I think)

In [ ]:
imaginery_data = [[1,2,3,4,5], [3,3,3,3,3]]

plt.scatter(imaginery_data[0],imaginery_data[1])


pca_test = PCA()
pca_test.fit(imaginery_data).explained_variance_ratio_

Here, the first principla component is responsible for all the variation. 'explained_variance_ratio_' should sum to one providing you're returning all the principle components. Any 2d data wiill have all the variance explained in the first component even if it is random 


In [ ]:
imaginary_data = np.random.rand(28,2)



plt.scatter(imaginary_data[:,0], imaginary_data[:,1])

print(imaginary_data[0:10])
pca = PCA(n_components = 1)
pca.fit(imaginary_data).explained_variance_ratio_

In [ ]:
imaginary_data_pca = pca.transform(imaginary_data)

print("original shape:   ", imaginary_data.shape)
print("transformed shape:", imaginary_data_pca.shape)

In [ ]:
imaginary_data_pca_new = pca.inverse_transform(imaginary_data_pca)
plt.scatter(imaginary_data[:, 0], imaginary_data[:, 1], alpha=0.2)
plt.scatter(imaginary_data_pca_new[:, 0], imaginary_data_pca_new[:, 1], alpha=0.8)
plt.axis('equal')

We can see that the direction of max variance has been plotted by the principal components, if this were non random data it might be interesting

But what of 3d data

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
imaginary_data = np.random.rand(728,3)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

i = imaginary_data[:,0]
j = imaginary_data[:,1]
k = imaginary_data[:,2]



ax.scatter(i, j, k, c='r', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()



In [ ]:
pca = PCA(n_components=2) # project from 784 to 2 dimensions so we can view them 
pca.fit(imaginary_data)
imaginary_data_pca = pca.transform(imaginary_data)


In [ ]:
imaginary_data_pca_new = pca.inverse_transform(imaginary_data_pca)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

i = imaginary_data[:,0]
j = imaginary_data[:,1]
k = imaginary_data[:,2]



ax.scatter(i, j, k, c='r', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')


l = imaginary_data_pca_new[:,0]
m = imaginary_data_pca_new[:,1]


ax.scatter(l, m, c='b', marker='o')


plt.axis('equal')


Here you can see that random 3d data has been cast down to 2d data. Although, in this case it is rather pointless as the underlying data is random, we don't really learn anything.

let's do it with the image dataset

In [ ]:


pca = PCA(n_components=2) # project from 784 to 2 dimensions so we can view them 
principalComponents = pca.fit(X)

print('Explained variance for the 1st two components',principalComponents.explained_variance_ratio_)

principalComponents = pca.transform(X)
principal_df = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2'])
plt.scatter(principalComponents[:, 0], principalComponents[:, 1], s= 1, c=y_train, cmap='Spectral')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10));
plt.xlabel('PC1')
plt.ylabel('PC2')



Looking at the results from our PCA it would seem that reducing dimensions in this dataset will not be easy. As there is some seperation but not a lot.

To read more about PCA look here: https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html

What if we add are pca components to our knn model?

In [ ]:
data = {'Labels':labels, 'diagonals':diags, 'widths':widths, 'heights':heights, 'Area':frequencies, 'PC1':principalComponents[:, 0],'PC2':principalComponents[:, 1]}

df = pd.DataFrame(data)
df


In [ ]:
test_y = df['Labels']

test_x = df
test_x.drop('Labels', axis =1, inplace = True)

X_train1, X_val1, y_train1, y_val1 = train_test_split(test_x, test_y, test_size=0.1, random_state=1337)

clf_knn = KNeighborsClassifier()
clf_knn.fit(X_train1,y_train1)

accuracy_score(clf_knn.predict(X_val1), y_val1)

An improvement!

Next on my learning journey, we should surely look at some other dimensionality reduction tecniques.

1st up
#### Dictionary learning

From the skelarn website - https://scikit-learn.org/stable/modules/decomposition.html#decompositions

    Dictionary learning (DictionaryLearning) is a matrix factorization problem that amounts to finding a (usually overcomplete) dictionary that will perform well atsparsely encoding the fitted data.

    Representing data as sparse combinations of atoms from an overcomplete dictionary is suggested to be the way the mammalian primary visual cortex works. Consequently, dictionary learning applied on image patches has been shown to give good results in image processing tasks such as image completion, inpainting and denoising, as well as for supervised recognition tasks.

In [ ]:
from sklearn.decomposition import MiniBatchDictionaryLearning

mbdl = MiniBatchDictionaryLearning(n_components = 2)

mbdl.fit(X)

In [ ]:
comps = mbdl.transform(X)

In [ ]:
principal_df = pd.DataFrame(data = principalComponents, columns = ['C1', 'C2'])
plt.scatter(comps[:, 0], comps[:, 1], s= 1, c=y_train, cmap='Spectral')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10));
plt.xlabel('C1')
plt.ylabel('C2')

Interesting, it seems worse than PCA in terms of seperation. Does it anything to our model though?

In [ ]:
def test_model(d1,d1_lab, d2, d2_lab,data):
    

    data = {'Labels':labels, 'diagonals':diags, 'widths':widths, 'heights':heights, 'Area':frequencies, 'PC1':principalComponents[:, 0],'PC2':principalComponents[:, 1]}
    data[d1_lab] = d1
    data[d2_lab] = d2
    
    df = pd.DataFrame(data)
    
    print(df.columns)
    test_y = df['Labels']

    test_x = df
    test_x.drop('Labels', axis =1, inplace = True)

    X_train1, X_val1, y_train1, y_val1 = train_test_split(test_x, test_y, test_size=0.1, random_state=1337)

    clf_knn = KNeighborsClassifier()
    clf_knn.fit(X_train1,y_train1)

    return accuracy_score(clf_knn.predict(X_val1), y_val1)


test_model(comps[:, 0],'d1', comps[:, 1],'d2', data)

Woo, we are over 50%

for our next trick lets do LDA, in my research career, the one dimensionality reduction tecnique that comes after PCA is LDA

#### Linear Discriminant analysis

Linear Discriminant Analysis - https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

    A classifier with a linear decision boundary, generated by fitting class conditional densities to the data and using Bayes’ rule.

    The model fits a Gaussian density to each class, assuming that all classes share the same covariance matrix.

    The fitted model can also be used to reduce the dimensionality of the input by projecting it to the most discriminative directions.

SO it is a supervised method, that can be used to reduce dimensionality.I would guess, based on my expereince on kaggle that this will sperate the data better - and overfit

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components = 2, )
comps = lda.fit_transform(X,y_train.values)

In [ ]:
principal_df = pd.DataFrame(data = principalComponents, columns = ['C1', 'C2'])
plt.scatter(comps[:, 0], comps[:, 1], s= 1, c=y_train, cmap='Spectral')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10));
plt.xlabel('C1')
plt.ylabel('C2')

It does look better, slightly anyway. let's test it with out model

In [ ]:
test_model(comps[:, 0],'lda1', comps[:, 1],'lda2', data)

Oh, no we have regressed by using LDA - in this case. 

The next algorithm is completely new to me T-SNE, I have only seen this since being on kaggle so I no nothing about it other than alot of kernels with t-sne in the title rate quite highly.

#### distributed Stochastic Neighbor Embedding.
https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

t-SNE is a tool to visualize high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.

It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data) to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high. This will suppress some noise and speed up the computation of pairwise distances between samples.

If you understood that, please explain in the comments! I'm off to watch a youtube video

In [ ]:
'''from sklearn.manifold import TSNE

tsne = TSNE(n_components =2)
comps = tsne.fit_transform(X) 

principal_df = pd.DataFrame(data = principalComponents, columns = ['C1', 'C2'])
plt.scatter(comps[:, 0], comps[:, 1], s= 1, c=y_train, cmap='Spectral')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10));
plt.xlabel('C1')
plt.ylabel('C2')

'''



If you want to see an impressive 3d t-sne visualisation look here https://projector.tensorflow.org/

In [ ]:
#test_model(comps[:, 0],'tsne1', comps[:, 1],'tsne2', data)#

A massive improvement almost 80% accurate now. Although it did take about 30 minutes to run.

<a name="section-three"></a>
## Building the CNN model

In [ ]:


# Normalize the data
X = X / 255.0
test = test / 255.0



In [ ]:
# re-shaping the data so that keras can use it, this is something that trips me up every time

X = X.values.reshape(X.shape[0], 28, 28,1)
test = test.values.reshape(test.shape[0], 28, 28,1)



In [ ]:
# This modifies some images slightly, I have seen this in a few tutorials and it usually makes the model more accurate. As a beginner, it goes without saying I don't fully understand all the parameters

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
valid_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

In [ ]:
y_train = to_categorical(y_train,num_classes=10) # the labels need to be one-hot encoded, this is something else I usually forget



In [ ]:
# function to modify the learning rate, if the loss does not change. Also saving the best weights of the model for prediction.

learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', 
                                            patience=3, 
                                            verbose=10, 
                                            factor=0.5, 
                                            min_lr=0.00001)



In [ ]:
# Using this notebook as a guide https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist if you haven't read it, and you want to learn about cnn's do yourself a favour and read it

def build_model(input_shape=(28, 28, 1), classes = 10):
    
    activation = 'relu'
    padding = 'same'
    gamma_initializer = 'uniform'
    
    input_layer = Input(shape=input_shape)
    
    hidden=Conv2D(32, (3,3), padding=padding,activation = activation, name="conv1")(input_layer)
    hidden=BatchNormalization(name="batch1")(hidden)
    hidden=Conv2D(32, (3,3), padding=padding,activation = activation, name="conv2")(hidden)
    hidden=BatchNormalization(name="batch2")(hidden)
    hidden=Conv2D(32, (5,5), padding=padding,activation = activation, name="conv3")(hidden)
    hidden=BatchNormalization(name="batch3")(hidden)
    hidden=MaxPool2D(pool_size=2, padding=padding, name="max1")(hidden)
    hidden=Dropout(0.4)(hidden)

    
    
    hidden=Conv2D(64, (3,3), padding =padding, activation = activation,  name="conv4")(hidden)
    hidden=BatchNormalization(name = 'batch4')(hidden)
    hidden=Conv2D(64, (3,3), padding =padding, activation = activation,  name="conv45")(hidden)
    hidden=BatchNormalization(name = 'batch5')(hidden)
    hidden=Conv2D(64, (5,5), padding =padding, activation = activation,  name="conv6")(hidden)
    hidden=BatchNormalization(name = 'batch6')(hidden)
    hidden=MaxPool2D(pool_size=2, padding="same", name="max2")(hidden)
    hidden=Dropout(0.4)(hidden)
    

    hidden=Flatten()(hidden)
    hidden=Dense(264,activation = activation, name="Dense1")(hidden)
    hidden=Dropout(0.3)(hidden)
    output = Dense(classes, activation = "softmax")(hidden)
    
    model = Model(inputs=input_layer, outputs=output)
    
    return model

In [ ]:
#keras.backend.clear_session()


In [ ]:
epochs = 50
initial_learningrate=2e-3
batch_size = 264

In [ ]:
# Define the optimizer
#optimizer = Adam(learning_rate=initial_learningrate, beta_1=0.9, beta_2=0.999, amsgrad=False)
optimizer = Adam(learning_rate=initial_learningrate)

# Compile the model


model = build_model(input_shape=(28, 28, 1), classes = 10)


model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])



In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size=0.1, random_state=1337)
datagen.fit(X_train)
valid_datagen.fit(X_val)


In [ ]:
callbacks = [learning_rate_reduction]
history = model.fit_generator(datagen.flow(X_train,y_train),
                              epochs = epochs,
                              validation_data=valid_datagen.flow(X_val,y_val),
                              verbose = 1,
                            callbacks = callbacks)

# On the first attempt I forgot to add the 'learning_rate_reduction'

#history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size ),
#                              epochs = epochs,
 #                             validation_data=valid_datagen.flow(X_val,y_val),
  #                            validation_steps = 50,
   #                           verbose = 1,
    #                          steps_per_epoch = X_train.shape[0] // batch_size,
     #                       callbacks = callbacks)



In [ ]:

model.summary()

<a name="section-four"></a>
## How did we do?

Interestingly for the first time, my notebook accuracy has been very similar to the leaderboard accuracy (version 9). That is encouraging! Now, that I am going to meddle with it I am sure it will change.
- v10: Doubled the dense layer and increased the epochs by 10

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best')

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best')

In [ ]:
import seaborn as sns

# used the code from https://www.kaggle.com/shahules/indian-way-to-learn-cnn to create this

y_pre_test=model.predict(X_val)
y_pre_test=np.argmax(y_pre_test,axis=1)
y_test=np.argmax(y_val,axis=1)

conf=confusion_matrix(y_test,y_pre_test)
conf=pd.DataFrame(conf,index=range(0,10),columns=range(0,10))


In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(conf, annot=True)

In [ ]:
print('out of {} samples, we got {} incorrect'.format(len(X_train), round(len(X_train) - history.history['accuracy'][-1] * len(X_train))))



I think the model was struggling with classifying the 1s and 7s du to rotation, so i removed rotation. maybe?

I am unsure why, but the results are very different to the leaderboard. 

In [ ]:

# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_val,axis = 1) 

# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 3
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize = (10,10))
    fig.tight_layout()
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 9 errors 
most_important_errors = sorted_dela_errors[-9:]

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

In [ ]:
predictions = model.predict(test)

In [ ]:
plt.imshow(test[0].reshape(28,28))

In [ ]:
predictions = predictions.argmax(axis = -1)
predictions

In [ ]:

submission['Label'] = predictions

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv',index=False)